# Mistral Modelleriyle Çalışmak

## Giriş

Bu derste şunları öğreneceksiniz:
- Farklı Mistral Modellerini keşfetmek
- Her modelin kullanım alanlarını ve senaryolarını anlamak
- Kod örnekleriyle her modelin kendine özgü özelliklerini göstermek


## Mistral Modelleri

Bu derste, 3 farklı Mistral modelini inceleyeceğiz: 
**Mistral Large**, **Mistral Small** ve **Mistral Nemo**.

Bu modellerin her biri Github Model pazarında ücretsiz olarak sunulmaktadır. Bu not defterindeki kod, bu modelleri kullanarak kodu çalıştıracaktır. Github Modelleri ile [AI modelleriyle prototipleme](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) hakkında daha fazla detaya buradan ulaşabilirsiniz.


## Mistral Large 2 (2407)
Mistral Large 2 şu anda Mistral'ın amiral gemisi modeli olup, kurumsal kullanım için tasarlanmıştır.

Model, orijinal Mistral Large'a göre şu geliştirmeleri sunar:
- Daha Geniş Bağlam Penceresi - 128k'ya karşı 32k
- Matematik ve Kodlama Görevlerinde Daha İyi Performans - %76,9 ortalama doğruluk oranı, önceki %60,4'e karşılık
- Artırılmış çok dilli performans - desteklenen diller: İngilizce, Fransızca, Almanca, İspanyolca, İtalyanca, Portekizce, Felemenkçe, Rusça, Çince, Japonca, Korece, Arapça ve Hintçe.

Bu özelliklerle birlikte, Mistral Large şu alanlarda öne çıkar:
- *Retrieval Augmented Generation (RAG)* - daha geniş bağlam penceresi sayesinde
- *Function Calling* - bu modelde yerleşik fonksiyon çağırma özelliği bulunur ve bu sayede harici araçlar ve API'lerle entegrasyon sağlanabilir. Bu çağrılar hem paralel hem de ardışık olarak yapılabilir.
- *Kod Üretimi* - bu model Python, Java, TypeScript ve C++ kod üretiminde oldukça başarılıdır.


Bu örnekte, bir metin belgesi üzerinde RAG deseni çalıştırmak için Mistral Large 2 kullanıyoruz. Soru Korece yazılmış ve yazarın üniversite öncesi faaliyetleri hakkında bilgi istiyor.

Metin belgesinin ve sorunun gömülerini oluşturmak için Cohere Embeddings Model kullanılıyor. Bu örnekte, vektör deposu olarak faiss Python paketi kullanılmıştır.

Mistral modeline gönderilen istemde hem sorular hem de soruya benzer olan alınan parçalar yer alıyor. Model daha sonra doğal bir dilde yanıt veriyor.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small, Mistral model ailesinin premier/kurumsal kategorisindeki bir diğer modelidir. Adından da anlaşılacağı gibi, bu model Küçük Dil Modeli (SLM) olarak tasarlanmıştır. Mistral Small kullanmanın avantajları şunlardır: 
- Mistral Large ve NeMo gibi Mistral LLM’lere kıyasla maliyet tasarrufu sağlar – %80’e varan fiyat düşüşü
- Düşük gecikme süresi – Mistral’ın LLM’lerine göre daha hızlı yanıt verir
- Esnek – farklı ortamlarda, gerekli kaynaklar konusunda daha az kısıtlamayla kullanılabilir. 

Mistral Small şu alanlarda oldukça uygundur: 
- Özetleme, duygu analizi ve çeviri gibi metin tabanlı görevler
- Maliyet avantajı sayesinde sık sık istek yapılan uygulamalar
- Kod inceleme ve önerisi gibi düşük gecikmeli kod görevleri


## Mistral Small ve Mistral Large Karşılaştırması

Mistral Small ve Large arasındaki gecikme farklarını göstermek için aşağıdaki hücreleri çalıştırın.

Yanıt sürelerinde 3-5 saniyelik bir fark görmelisiniz. Ayrıca aynı istemde yanıt uzunluklarını ve tarzını da gözlemleyin.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Bu derste ele alınan diğer iki modele kıyasla, Mistral NeMo Apache2 Lisansı ile sunulan tek ücretsiz modeldir.

Mistral'ın önceki açık kaynak LLM'i olan Mistral 7B'nin bir üst versiyonu olarak görülmektedir.

NeMo modelinin diğer bazı özellikleri şunlardır:

- *Daha verimli tokenizasyon:* Bu model, yaygın olarak kullanılan tiktoken yerine Tekken tokenlaştırıcısını kullanır. Bu sayede daha fazla dil ve kod üzerinde daha iyi performans sağlar.

- *İnce ayar yapabilme:* Temel model, ince ayar yapılabilir şekilde sunulmaktadır. Bu da ince ayarın gerekebileceği kullanım senaryoları için daha fazla esneklik sağlar.

- *Yerel Fonksiyon Çağrısı* - Mistral Large gibi, bu model de fonksiyon çağrısı konusunda eğitilmiştir. Bu özelliğiyle, bunu yapan ilk açık kaynak modellerden biri olarak öne çıkar.


## Mistral NeMo

Bu derste ele alınan diğer iki modele kıyasla, Mistral NeMo Apache2 Lisansı ile sunulan tek ücretsiz modeldir.

Mistral'ın önceki açık kaynak LLM'i olan Mistral 7B'nin bir üst versiyonu olarak görülmektedir.

NeMo modelinin diğer bazı özellikleri şunlardır:

- *Daha verimli tokenizasyon:* Bu model, yaygın olarak kullanılan tiktoken yerine Tekken tokenlaştırıcısını kullanır. Bu sayede daha fazla dil ve kod üzerinde daha iyi performans sağlar.

- *İnce ayar (Finetuning):* Temel model, ince ayar yapılabilir şekilde sunulmaktadır. Bu da ince ayarın gerekebileceği kullanım senaryoları için daha fazla esneklik sağlar.

- *Yerel Fonksiyon Çağrısı* - Mistral Large gibi, bu model de fonksiyon çağrısı konusunda eğitilmiştir. Bu özelliğiyle, bunu yapan ilk açık kaynak modellerden biri olarak öne çıkar.


### Tokenizer'ları Karşılaştırma

Bu örnekte, Mistral NeMo'nun tokenizasyonu Mistral Large ile nasıl karşılaştırdığına bakacağız.

Her iki örnek de aynı girdiyi alıyor, ancak NeMo'nun Mistral Large'a göre daha az token döndürdüğünü göreceksiniz.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Öğrenme burada bitmiyor, yolculuğa devam et

Bu dersi tamamladıktan sonra, Generative AI bilginizi geliştirmeye devam etmek için [Generative AI Öğrenme koleksiyonumuza](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) göz atmayı unutmayın!



---

**Feragatname**:
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerde hata veya yanlışlıklar bulunabileceğini lütfen unutmayın. Belgenin orijinal diliyle hazırlanmış hali esas alınmalıdır. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından doğabilecek yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
